In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import fitz  # PyMuPDF

# Função para extrair imagens de um arquivo PDF usando PyMuPDF
def extract_images_from_pdf(pdf_path):
    images = []
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap()
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, pix.n))
        images.append(img)
    doc.close()
    return images

# Carregar o modelo treinado
model = load_model('modelo_erros_acertos.h5')

# Pasta de teste contendo arquivos PDF
pasta_teste = 'datasets/teste'

# Lista para armazenar resultados
results = []

# Iterar sobre os arquivos na pasta de teste
for filename in os.listdir(pasta_teste):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pasta_teste, filename)
        
        # Extrair imagens do PDF
        images = extract_images_from_pdf(pdf_path)
        
        # Processar cada imagem e fazer a previsão
        for img in images:
            img = tf.image.resize(img, (150, 150))  # Redimensionar para o tamanho esperado
            img_array = np.expand_dims(img, axis=0) / 255.0  # Normalizar e adicionar dimensão do lote
            
            # Fazer a previsão
            prediction = model.predict(img_array)
            predicted_class = int(np.round(prediction[0]))
            class_label = 'acertos' if predicted_class == 1 else 'erros'
            
            # Armazenar o resultado
            results.append((filename, class_label))

# Mostrar resultados
for result in results:
    print(f'Arquivo: {result[0]}, Classificação: {result[1]}')
